In [2]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable 



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [5]:
data_path = '/content/drive/MyDrive/비트트레이더/데이터'
train_x_df = pd.read_csv(data_path  + "/train_x_df.csv")
train_y_df = pd.read_csv(data_path  + "/train_y_df.csv")
test_x_df = pd.read_csv(data_path  + "/test_x_df.csv")

## Data preprocessing

input : (seq_len, batch, input_size)

In [6]:
col_name=list(train_x_df.columns)

In [7]:
test_x_df.iloc[:, 3:]

,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,1.003541,1.007110,1.003541,1.006090,486928.968750,182543.687500,332.671570,287449.968750,107758.375000
1,1.006117,1.006331,1.004829,1.005634,824485.375000,309019.750000,407.790955,560290.250000,210005.125000
2,1.005929,1.005983,1.004266,1.005366,478614.562500,179301.843750,276.332031,214270.265625,80283.554688
3,1.005044,1.005956,1.004910,1.005795,334366.843750,125305.117188,166.335785,56401.246094,21135.490234
4,1.005580,1.006466,1.005285,1.005929,560916.687500,210316.796875,303.160370,138711.703125,52010.964844
...,...,...,...,...,...,...,...,...,...
730015,1.001805,1.001805,0.998797,0.998797,583600.875000,77517.406250,473.898010,9092.824219,1210.823120
730016,0.998571,1.001730,0.996690,1.001730,688827.312500,91420.523438,503.986786,289617.906250,38465.292969
730017,0.999323,1.003009,0.997367,1.002934,417688.437500,55624.304688,353.542969,291073.437500,38773.152344
730018,1.000903,1.000903,0.999774,0.999925,91273.515625,12139.500000,120.355049,0.000000,0.000000


In [272]:
test_x_df[test_x_df['sample_id']==0]

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,7,1.003541,1.007110,1.003541,1.006090,486928.96875,182543.687500,332.671570,287449.968750,107758.375000
1,0,1,7,1.006117,1.006331,1.004829,1.005634,824485.37500,309019.750000,407.790955,560290.250000,210005.125000
2,0,2,7,1.005929,1.005983,1.004266,1.005366,478614.56250,179301.843750,276.332031,214270.265625,80283.554688
3,0,3,7,1.005044,1.005956,1.004910,1.005795,334366.84375,125305.117188,166.335785,56401.246094,21135.490234
4,0,4,7,1.005580,1.006466,1.005285,1.005929,560916.68750,210316.796875,303.160370,138711.703125,52010.964844
...,...,...,...,...,...,...,...,...,...,...,...,...
1375,0,1375,7,0.997398,0.999115,0.997371,0.998256,411115.00000,152972.437500,324.623047,221016.265625,82244.789062
1376,0,1376,7,0.998256,0.999624,0.998256,0.998873,656922.50000,244636.890625,448.033478,395109.750000,147137.734375
1377,0,1377,7,0.999168,0.999839,0.999168,0.999785,328069.96875,122220.281250,222.675323,215481.843750,80277.742188
1378,0,1378,7,0.999624,1.000510,0.999598,1.000000,301314.87500,112317.312500,321.940216,130979.765625,48832.816406


In [270]:
train_y_df[train_y_df['sample_id']==0]

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,7,1.000392,1.000588,1.000392,1.000588,8.305119e+05,42356.179688,137.308746,8.305119e+05,42356.179688
1,0,1,7,1.000588,1.001177,1.000392,1.001177,5.320066e+05,27140.638672,294.233032,4.882738e+05,24909.861328
2,0,2,7,1.001177,1.001177,1.001177,1.001177,5.113770e+05,26100.681641,58.846603,5.113770e+05,26100.681641
3,0,3,7,1.001177,1.001373,1.001177,1.001373,1.134853e+06,57929.410156,137.308746,1.095514e+06,55921.156250
4,0,4,7,1.000981,1.000981,0.999804,1.000196,5.801173e+06,295872.343750,666.928162,9.911239e+05,50528.589844
...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,115,7,1.002354,1.002942,1.001569,1.001569,2.640449e+06,134901.953125,666.928162,2.410357e+05,12318.685547
116,0,116,7,1.001569,1.001569,1.000981,1.000981,9.847371e+05,50265.953125,137.308746,1.467634e+05,7493.740723
117,0,117,7,1.001373,1.001373,1.000785,1.000785,7.983522e+03,407.440552,39.231071,4.040800e+03,206.282852
118,0,118,7,1.001373,1.002746,1.001373,1.002550,3.620155e+06,184873.296875,568.850525,3.505575e+06,179018.250000


In [334]:
X_train_tensors = Variable(torch.Tensor(np.array(train_x_df.iloc[:, 3:])))

X_test_tensors = Variable(torch.Tensor(np.array(test_x_df.iloc[:, 3:])))

y_train_tensors = Variable(torch.Tensor(np.array(train_y_df.iloc[:, 4:5])))

# X_train_tensors = Variable(torch.Tensor(np.array(train_x_df[train_x_df.loc[:, col_name[3:]]])))

# X_test_tensors = Variable(torch.Tensor(np.array(test_x_df[test_x_df.loc[:, col_name[3:]]])))

# y_train_tensors = Variable(torch.Tensor(np.array(train_y_df[train_y_df.iloc[:, 3:])))

In [202]:
X_train_tensors.shape, X_test_tensors.shape, y_train_tensors.shape

(torch.Size([10159560, 9]), torch.Size([730020, 9]), torch.Size([883440, 1]))

In [342]:
train_y_df

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,7,1.000392,1.000588,1.000392,1.000588,8.305119e+05,42356.179688,137.308746,8.305119e+05,42356.179688
1,0,1,7,1.000588,1.001177,1.000392,1.001177,5.320066e+05,27140.638672,294.233032,4.882738e+05,24909.861328
2,0,2,7,1.001177,1.001177,1.001177,1.001177,5.113770e+05,26100.681641,58.846603,5.113770e+05,26100.681641
3,0,3,7,1.001177,1.001373,1.001177,1.001373,1.134853e+06,57929.410156,137.308746,1.095514e+06,55921.156250
4,0,4,7,1.000981,1.000981,0.999804,1.000196,5.801173e+06,295872.343750,666.928162,9.911239e+05,50528.589844
...,...,...,...,...,...,...,...,...,...,...,...,...
883435,7361,115,8,1.007303,1.007845,1.007261,1.007720,2.843995e+00,686.734558,0.792856,1.373841e+00,331.730499
883436,7361,116,8,1.007678,1.008847,1.007678,1.008471,2.577599e+00,622.798462,0.884660,1.638966e+00,396.005463
883437,7361,117,8,1.008513,1.009014,1.008304,1.008805,4.579865e+00,1106.928345,1.210149,1.735705e+00,419.485596
883438,7361,118,8,1.008680,1.009222,1.008680,1.009222,1.065129e+00,257.542755,0.525789,4.348240e-01,105.134232


In [337]:
#reshaping to rows, timestamps, features

X_train_tensors_final = torch.reshape(X_train_tensors,   (-1, 1380, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (-1 , 1380, X_test_tensors.shape[1])) 
y_train_tensors_final =torch.reshape(y_train_tensors,  (-1 , 120, y_train_tensors.shape[1])) 
# X_train_tensors_final = torch.reshape(train_x_df.iloc[:,3].values,   (-1, 1380, 1))


In [338]:
X_train_tensors_final.shape

torch.Size([7362, 1380, 9])

In [339]:
y_train_tensors_final.shape

torch.Size([7362, 120, 1])

In [340]:
X_train_tensors_final.shape

torch.Size([7362, 1380, 9])

In [341]:
X_test_tensors_final.shape[1]

1380

In [310]:
X_train_data_loader = torch.utils.data.DataLoader(X_train_tensors_final, 690, shuffle=False)
y_train_data_loader = torch.utils.data.DataLoader(y_train_tensors_final, 1227, shuffle=False)
X_test_data_loader = torch.utils.data.DataLoader(X_test_tensors_final, 1227, shuffle=False)


In [311]:
for batch in X_train_data_loader:
    X_train_data = batch

for batch in y_train_data_loader:
    y_train_data = batch

In [220]:
from torch.utils.data import TensorDataset # 텐서데이터셋


In [252]:
dataset = TensorDataset(X_train_tensors_final, y_train_tensors_final)

In [222]:
dataloader = DataLoader(dataset, batch_size=1380, shuffle=False)

In [225]:
x_train.shape, y_train.shape

(torch.Size([462, 1380, 9]), torch.Size([462, 120, 1]))

In [312]:
y_train_data.shape, X_train_data.shape

(torch.Size([1227, 60, 1]), torch.Size([1227, 60, 9]))

# LSTM modeling

In [325]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 120) #fully connected 1
        self.fc = nn.Linear(120, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [326]:
 nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                          num_layers=num_layers, batch_first=True) 

LSTM(9, 10, batch_first=True)

In [327]:
 X_train_data.shape

torch.Size([1227, 60, 9])

In [328]:
X_train_tensors_final.shape

torch.Size([169326, 60, 9])

In [329]:
Variable(torch.zeros(1, 462, 9)) .shape

torch.Size([1, 462, 9])

In [330]:
num_epochs = 1000 #1000 epochs
learning_rate = 0.1 #0.01 lr

input_size = 9 #number of features
# batch_size = 1380 # batch size
hidden_size = 10 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 

In [331]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, 1) #our lstm class 

In [332]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

In [333]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    outputs = lstm1.forward(X_train_data) #forward pass
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    
    # obtain the loss function
    loss = criterion(outputs, y_train_data)
    
    loss.backward() #calculates the loss of the loss function
    
    optimizer.step() #improve from loss, i.e backprop
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1227, 60, 1])) that is different to the input size (torch.Size([1227, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored

In [ ]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(X_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors_final)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

In [ ]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast())

In [ ]:
train_y_df

- 참조 : https://cnvrg.io/pytorch-lstm/?gclid=CjwKCAjwgOGCBhAlEiwA7FUXknvaSTYr_3NHdJqAuctM0aDmets4r7K9YyLoHXhe41Bh6RRooaYxYRoCuxAQAvD_BwE 